In [54]:
! pip install cpi



In [55]:
import cpi
cpi.update()

In [78]:
import pandas as pd 
import numpy as np
from datetime import date

In [79]:

import matplotlib.pyplot as plt

# Data Wrangling Pitcher Dataset

In [80]:
#reading in pitcher only dataset
df_pitcher = pd.read_csv('df_pitcher_full.csv')

#creating copy of original dataframe
df_pitcher_org = df_pitcher.copy()

In [81]:
#removing position players 
indexNames_p = df_pitcher[df_pitcher['posit'] != 'P' ].index
df_pitcher.drop(indexNames_p, inplace=True)

#fullname player column
df_pitcher['playerName'] = df_pitcher["firstName"] +" "+ df_pitcher["lastName"]

#drop useless columns
df_pitcher.drop(['mlbid', 'retroid', 'playerid'], axis=1, inplace=True)

#change 'borndate' column to datetime object
df_pitcher['borndate'] = pd.to_datetime(df_pitcher['borndate'])

#converting 'Age' column datatype to numeric
df_pitcher['Age'] = pd.to_numeric(df_pitcher['Age'])

#creating a new salary column
df_pitcher['salary_filled'] = df_pitcher['salary']


df_pitcher['yearDt'] = pd.to_datetime(df_pitcher['year'])
df_pitcher['yearDt'] = df_pitcher['yearDt'].dt.year

# Missing Values

In [82]:
#checking for percentage of missing data
df_pitcher.isnull().sum()/len(df_pitcher)

#replacing salary null values with league minimum for respective years
salary_dict = {2019:555000, 2018:545000, 2017:535000, 2016:507500, 2015:507500, 2014:480000, 
               2013:480000, 2012:480000, 2011:414000, 2010:400000, 2009:400000, 2008:390000, 
               2007:380000, 2006:327000, 2005:316000, 2004:300000, 2003:300000, 2002:300000,
               2001:200000, 2000:200000, 1999:109000, 1998:109000, 1997:109000, 1996:109000,
               1995:109000, 1994:100000, 1993:100000, 1992:100000, 1991:100000, 1990:100000}

df_pitcher['salary_filled'] = df_pitcher['salary'].fillna(df_pitcher['year'].map(salary_dict))

In [83]:
#add flag column
df_pitcher['flag'] = [0 if x > 0 else 1 for x in df_pitcher['salary']]

In [84]:
#adjust salary for inflation
df_pitcher['adj_salary'] = df_pitcher.apply(lambda x: cpi.inflate(x.salary, x.yearDt), axis=1)
df_pitcher['adj_salary_filled'] = df_pitcher.apply(lambda x: cpi.inflate(x.salary_filled, x.yearDt), axis=1)
df_pitcher['adj_salary'] = df_pitcher['adj_salary'].round(1)
df_pitcher['adj_salary_filled'] = df_pitcher['adj_salary_filled'].round(1)

In [85]:
#reorder columns
col_new_order = ['playerName','salary', 'salary_filled', 'adj_salary', 'adj_salary_filled', 'flag','Age', 'HT', 'WT', 'Bats', 'Throws','year', 'teamName','posit', 'borndate', 'Place', 'LeagueAbbr', 'W', 'L', 'G', 'GS', 'CG', 'SHO', 'GF', 'SV', 'IP', 'H', 'HR', 'R', 'ER', 'BB','IBB', 'SO', 'WP', 'BK', 'ERA', 'h9', 'hr9', 'bb9', 'so9', 'WHIP']
df_pitcher = df_pitcher.reindex(columns=col_new_order)

# Feature Engineering

In [86]:
#add column with number of year in MLB 
df_pitcher_final = df_pitcher.sort_values(by=['playerName', 'year'])
df_pitcher_final['year'] = df_pitcher_final['year'].astype(str)
df_pitcher_final['total_years_mlb'] = df_pitcher_final.groupby('playerName').cumcount()+1
#add earned minimum salary for player year in MLB 
df_pitcher_final['minimum_year'] = df_pitcher_final['year'].apply(lambda x: salary_dict[int(x)])

In [87]:
#drop useless columns
df_pitcher_final.drop(['adj_salary', 'salary_filled'], axis=1, inplace=True)

#use player data going back until the 2010 season 
df_pitcher_final = df_pitcher_final[df_pitcher_final['year'] >= '2010'] 
df_pitcher_final

,playerName,salary,adj_salary_filled,flag,Age,HT,WT,Bats,Throws,year,...,WP,BK,ERA,h9,hr9,bb9,so9,WHIP,total_years_mlb,minimum_year
18081,AJ Achter,NaN,3106478.4,1,25,6-5,190,R,R,2014,...,0,0,3.27,11.45,1.64,2.45,4.09,1.55,1,480000
18082,AJ Achter,507500.0,3284453.7,0,26,6-5,190,R,R,2015,...,0,0,6.75,8.10,2.70,4.05,9.45,1.35,2,507500
18083,AJ Achter,NaN,3284453.7,1,27,6-5,190,R,R,2016,...,0,0,3.11,10.27,1.67,2.87,3.35,1.46,3,507500
3243,AJ Burnett,16500000.0,106785193.3,0,33,6-5,205,R,R,2010,...,16,0,5.26,9.84,1.21,3.76,6.99,1.51,12,400000
3244,AJ Burnett,16500000.0,106785193.3,0,34,6-5,205,R,R,2011,...,25,0,5.15,8.98,1.47,3.92,8.18,1.43,13,414000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18343,Zack Wheeler,800000.0,5177463.9,0,27,6-3,180,R,R,2017,...,1,0,5.21,10.11,1.56,4.17,8.44,1.59,3,535000
18344,Zack Wheeler,1900000.0,12296476.8,0,28,6-3,180,R,R,2018,...,2,1,3.31,7.40,0.69,2.71,8.84,1.12,4,545000
18345,Zack Wheeler,5975000.0,38669183.6,0,29,6-3,180,R,R,2019,...,5,0,3.96,9.03,1.01,2.30,8.98,1.26,5,555000
17696,Zeke Spruill,NaN,3106478.4,1,23,6-4,184,B,R,2013,...,0,0,5.56,13.50,2.38,3.97,7.15,1.94,1,480000


# Data Wrangling Hitter Dataset

In [88]:
#reading in hitter only dataset 
df_hitter = pd.read_csv('df_hitter.csv')

#creating copy of original dataframe
df_hitter_org = df_hitter.copy()

In [89]:
#removing pitchers
df_hitter.drop(df_hitter[(df_hitter['posit'].values == ['P']) | (df_hitter['posit'].values == ['p'])].index, inplace=True)

#fullname player column
df_hitter['playerName'] = df_hitter["firstName"] +" "+ df_hitter["lastName"]

#drop useless columns
df_hitter.drop(['mlbid', 'retroid', 'playerid'], axis=1, inplace=True)

#change 'borndate' column to datetime object
df_hitter['borndate'] = pd.to_datetime(df_hitter['borndate'])

#converting 'Age' column datatype to numeric
df_hitter['Age'] = pd.to_numeric(df_hitter['Age'])

In [90]:
#checking for percentage of missing data
df_hitter.isnull().sum()/len(df_hitter)

#creating a new salary column
df_hitter['salary_filled'] = df_hitter['salary']

#replacing salary null values with league minimum for respective years
df_hitter['salary_filled'] = df_hitter['salary'].fillna(df_hitter['year'].map(salary_dict))

# Adjusting for Inflation

In [93]:
df_hitter['yearDt'] = pd.to_datetime(df_hitter['year'])
df_hitter['yearDt'] = df_hitter['yearDt'].dt.year

In [94]:
#add flag column
df_hitter['flag'] = [0 if x > 0 else 1 for x in df_hitter['salary']]

#adjust salary for inflation
df_hitter['adj_salary'] = df_hitter.apply(lambda x: cpi.inflate(x.salary, x.yearDt), axis=1)
df_hitter['adj_salary_filled'] = df_hitter.apply(lambda x: cpi.inflate(x.salary_filled, x.yearDt), axis=1)
df_hitter['adj_salary'] = df_hitter['adj_salary'].round(1)
df_hitter['adj_salary_filled'] = df_hitter['adj_salary_filled'].round(1)

In [95]:
#reorder columns
col_new_order_hitter = ['playerName','salary', 'salary_filled', 'adj_salary', 'adj_salary_filled', 'flag', 'Age', 'HT',
       'WT', 'Bats', 'Throws', 'posit', 'borndate', 'Place','teamName', 'LeagueAbbr', 'G', 'AB',
       'R', 'H', 'Dbl', 'Tpl', 'HR', 'RBI', 'SB', 'CS', 'BB', 'IBB', 'SO',
       'SH', 'SF', 'HBP', 'GDP', 'Bavg', 'Slg', 'obp', 'OPS', 'year']
df_hitter = df_hitter.reindex(columns=col_new_order_hitter)

In [96]:
#add column with number of year in MLB 
df_hitter_final = df_hitter.sort_values(by=['playerName', 'year'])
df_hitter_final['year'] = df_hitter_final['year'].astype(str)
df_hitter_final['total_years_mlb'] = df_hitter_final.groupby('playerName').cumcount()+1

#add earned minimum salary for player year in MLB 
df_hitter_final['minimum_year'] = df_hitter_final['year'].apply(lambda x: salary_dict[int(x)])

In [97]:
#drop useless columns
df_hitter_final.drop(['adj_salary', 'salary_filled'], axis=1, inplace=True)

#use player data going back until the 2010 season 
df_hitter_final = df_hitter_final[df_hitter_final['year'] >= '2010'] 
df_hitter_final

,playerName,salary,adj_salary_filled,flag,Age,HT,WT,Bats,Throws,posit,...,SF,HBP,GDP,Bavg,Slg,obp,OPS,year,total_years_mlb,minimum_year
20831,AJ Ellis,NaN,2588732.0,1,29,6-3,240,R,R,C,...,1,1,5,0.278,0.324,0.363,0.687,2010,3,400000
20832,AJ Ellis,421000.0,2724640.4,0,30,6-3,240,R,R,c,...,0,3,2,0.271,0.376,0.392,0.768,2011,4,414000
20833,AJ Ellis,490000.0,3171196.6,0,31,6-3,240,R,R,C,...,4,7,17,0.270,0.414,0.373,0.787,2012,5,480000
20834,AJ Ellis,2000000.0,12943659.8,0,32,6-3,240,R,R,C,...,6,3,11,0.238,0.364,0.318,0.682,2013,6,480000
20835,AJ Ellis,3550000.0,22974996.1,0,33,6-3,240,R,R,C,...,4,4,15,0.191,0.254,0.323,0.577,2014,7,480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24429,Zack Cozart,12666667.0,81976514.2,0,33,6-1,185,R,R,3B,...,3,2,2,0.124,0.144,0.178,0.322,2019,9,555000
29148,Zack Granite,NaN,3462429.0,1,24,6-0,175,L,L,CF,...,1,0,6,0.237,0.290,0.321,0.611,2017,1,535000
26186,Zelous Wheeler,NaN,3106478.4,1,27,5-10,220,R,R,3B,...,1,1,1,0.193,0.298,0.230,0.528,2014,1,480000
26411,Zoilo Almonte,NaN,3106478.4,1,24,5-11,165,B,R,LF,...,1,0,2,0.236,0.302,0.274,0.576,2013,1,480000


In [98]:
# #saving cleaned dataframes 
df_pitcher_final.to_csv('df_pitcher_processed.csv', index=False)
df_hitter_final.to_csv('df_hitter_processed.csv', index=False)